# Определение цвета товара по фотографии

## Предварительные требования (Prerequisites)

Для запуска этого ноутбука требуются следующие зависимости:

```
Python >= 3.10
numpy >= 1.24.3
pandas >= 2.0.3
torch >= 2.0.1
torchvision >= 0.15.2
timm >= 0.9.7
scikit-learn >= 1.3.0
tqdm >= 4.66.1
Pillow >= 10.0.0
matplotlib >= 3.7.2
```

Вы можете установить их с помощью pip:
```bash
pip install numpy pandas torch torchvision timm scikit-learn tqdm Pillow matplotlib
```

Также необходимо иметь:
1. Файл с весами модели (будет загружен автоматически)
2. Изображение товара для анализа

В данном ноутбуке мы будем использовать модель для определения цвета товара по его фотографии.

In [ ]:
import os
import sys
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.colors import to_rgb
from PIL import Image
import torch
import torch.nn as nn
from torchvision import transforms
from IPython.display import display, HTML
import ipywidgets as widgets
from pathlib import Path

# Добавляем директорию с initialize_model в путь поиска модулей
# Если скрипт находится в другой директории, скорректируйте путь
sys.path.append(os.path.abspath('./'))
sys.path.append(os.path.abspath('../'))

try:
    from initialize_model import initialize_model, COLORS
except ImportError:
    from notebooks.initialize_model import initialize_model, COLORS

## Загрузка модели

Загрузим предобученную модель для определения цвета товара. Если весов модели нет на диске, они будут загружены автоматически.

In [ ]:
# URL для загрузки весов модели
WEIGHTS_URL = "https://drive.usercontent.google.com/download?id=17p0eXDdtpcpHF4kmApanexzqGWFFXdir&export=download&authuser=0&confirm=t&uuid=d335c718-4016-44be-8212-f8846e1bc333&at=AEz70l75v8L-U5sRzAy--ChfoK6D%3A1741921834963"

# Путь для сохранения весов
WEIGHTS_PATH = "../weights/model_weights.pth"
if not os.path.exists(os.path.dirname(WEIGHTS_PATH)):
    os.makedirs(os.path.dirname(WEIGHTS_PATH), exist_ok=True)

# Инициализация модели
model = initialize_model(WEIGHTS_PATH, WEIGHTS_URL)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Модель успешно загружена и работает на устройстве: {device}")

## Функция предсказания цвета

Создадим функцию для предсказания цвета товара по его изображению и категории.

In [ ]:
# Преобразования для изображения
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Словарь соответствия категорий
category_to_idx = {
    'одежда_д': 0,
    'столы': 1,
    'стулья': 2,
    'сумки': 3
}

# Обратный словарь для удобства
idx_to_category = {v: k for k, v in category_to_idx.items()}

# Цветовой код для визуализации
color_code = {
    'бежевый': '#F5F5DC',
    'белый': '#FFFFFF',
    'бирюзовый': '#40E0D0',
    'бордовый': '#800000',
    'голубой': '#87CEEB',
    'желтый': '#FFFF00',
    'зеленый': '#008000',
    'золотой': '#FFD700',
    'коричневый': '#8B4513',
    'красный': '#FF0000',
    'оранжевый': '#FFA500',
    'разноцветный': '#FFFFFF',  # Для разноцветного используем белый фон
    'розовый': '#FFC0CB',
    'серебряный': '#C0C0C0',
    'серый': '#808080',
    'синий': '#0000FF',
    'фиолетовый': '#800080',
    'черный': '#000000'
}

def predict_color(image_path, category, model, device, transform, top_k=5):
    """Предсказание цвета товара по изображению и категории
    
    Args:
        image_path (str): Путь к изображению
        category (str): Категория товара (одежда_д, столы, стулья, сумки)
        model (nn.Module): Модель для предсказания
        device (torch.device): Устройство для вычислений
        transform (transforms.Compose): Преобразования для изображения
        top_k (int): Количество лучших результатов для вывода
        
    Returns:
        tuple: Кортеж из лучшего предсказания и списка top_k предсказаний с вероятностями
    """
    # Проверка валидности категории
    if category not in category_to_idx:
        raise ValueError(f"Недопустимая категория: {category}. Доступные категории: {list(category_to_idx.keys())}")
    
    # Загрузка и предобработка изображения
    try:
        image = Image.open(image_path).convert('RGB')
    except Exception as e:
        raise Exception(f"Ошибка при загрузке изображения {image_path}: {e}")
    
    # Подготовка данных для модели
    img_tensor = transform(image).unsqueeze(0).to(device)
    category_idx = torch.tensor([category_to_idx[category]]).to(device)
    
    # Прогноз модели
    with torch.no_grad():
        outputs = model(img_tensor, category_idx)
        probabilities = torch.softmax(outputs, dim=1)[0]
    
    # Преобразование результатов
    probs_np = probabilities.cpu().numpy()
    sorted_indices = np.argsort(probs_np)[::-1]
    
    # Список русских названий цветов
    color_names = list(COLORS.keys())
    
    # Получаем лучшее предсказание и список top-k предсказаний
    best_color = color_names[sorted_indices[0]]
    top_k_predictions = [
        (color_names[idx], float(probs_np[idx]))
        for idx in sorted_indices[:top_k]
    ]
    
    return best_color, top_k_predictions

def visualize_prediction(image_path, category, best_color, top_k_predictions):
    """Визуализация предсказания цвета для изображения
    
    Args:
        image_path (str): Путь к изображению
        category (str): Категория товара
        best_color (str): Лучшее предсказание цвета
        top_k_predictions (list): Список кортежей (цвет, вероятность) для top-k предсказаний
    """
    # Загрузка изображения
    image = Image.open(image_path).convert('RGB')
    
    # Создание фигуры с двумя подграфиками
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 6))
    
    # Отображение изображения
    ax1.imshow(image)
    ax1.set_title(f"Изображение товара: {category}")
    ax1.axis('off')
    
    # Отображение гистограммы вероятностей
    colors = [color for color, _ in top_k_predictions]
    probs = [prob * 100 for _, prob in top_k_predictions]  # В процентах
    bar_colors = [color_code.get(color, '#CCCCCC') for color in colors]
    
    y_pos = range(len(colors))
    ax2.barh(y_pos, probs, color=bar_colors)
    ax2.set_yticks(y_pos)
    ax2.set_yticklabels(colors)
    ax2.set_xlabel('Вероятность (%)')
    ax2.set_title('Вероятности цветов')
    ax2.set_xlim(0, 100)
    
    for i, v in enumerate(probs):
        ax2.text(v + 1, i, f"{v:.1f}%", va='center')
    
    # Информация о лучшем предсказании
    fig.suptitle(f"Предсказанный цвет: {best_color} ({probs[0]:.1f}%)")
    plt.tight_layout()
    plt.show()

## Пример использования модели

Сейчас мы протестируем модель на тестовом изображении. Для тестирования нужно указать путь к изображению и категорию товара.

In [ ]:
# Пример использования
image_path = "../test_images/example.jpg"  # Укажите путь к вашему изображению
category = "сумки"  # Укажите категорию товара (одежда_д, столы, стулья, сумки)

try:
    best_color, top_k_predictions = predict_color(
        image_path, category, model, device, transform, top_k=5
    )
    
    print(f"\nПредсказанный цвет: {best_color}")
    print("\nТоп-5 вероятностей:")
    for color, prob in top_k_predictions:
        print(f"{color}: {prob:.4f} ({prob*100:.1f}%)")
    
    # Визуализация результатов
    visualize_prediction(image_path, category, best_color, top_k_predictions)
except Exception as e:
    print(f"Ошибка при анализе изображения: {e}")
    print("\nВозможно, путь к файлу указан неверно или файл не существует.")
    print("Попробуйте использовать интерактивную форму ниже для загрузки изображения.")

## Интерактивная форма для определения цвета

Ниже представлена интерактивная форма, где вы можете ввести путь к изображению и выбрать категорию товара.

In [ ]:
# Создаем виджеты для интерактивного взаимодействия
image_path_widget = widgets.Text(
    value='../test_images/example.jpg',
    placeholder='Введите путь к изображению',
    description='Путь:',
    disabled=False
)

category_widget = widgets.Dropdown(
    options=list(category_to_idx.keys()),
    value='сумки',
    description='Категория:',
    disabled=False,
)

output_widget = widgets.Output()

def on_analyze_button_clicked(b):
    """Обработчик нажатия кнопки анализа"""
    with output_widget:
        output_widget.clear_output()
        try:
            image_path = image_path_widget.value
            category = category_widget.value
            
            print(f"Анализ изображения: {image_path}")
            print(f"Категория: {category}")
            
            best_color, top_k_predictions = predict_color(
                image_path, category, model, device, transform, top_k=5
            )
            
            print(f"\nПредсказанный цвет: {best_color}")
            print("\nТоп-5 вероятностей:")
            for color, prob in top_k_predictions:
                print(f"{color}: {prob:.4f} ({prob*100:.1f}%)")
            
            # Визуализация результатов
            visualize_prediction(image_path, category, best_color, top_k_predictions)
        except Exception as e:
            print(f"Ошибка при анализе изображения: {e}")

analyze_button = widgets.Button(
    description='Анализировать',
    disabled=False,
    button_style='success',
    tooltip='Нажмите для анализа изображения',
    icon='check'
)

analyze_button.on_click(on_analyze_button_clicked)

# Создаем макет для формы
form = widgets.VBox([
    widgets.HBox([image_path_widget, category_widget, analyze_button]),
    output_widget
])

display(form)

## Заключение

В этом ноутбуке мы продемонстрировали, как использовать предобученную модель для определения цвета товара по его фотографии.

Модель способна классифицировать товары на один из 18 возможных цветов с учетом категории товара (одежда для девочек, столы, стулья, сумки).

Для использования в своих проектах вы можете:
1. Загрузить данный ноутбук и весы модели
2. Адаптировать функцию `predict_color` для своих нужд
3. Интегрировать модель в свои приложения